In [78]:
#importações das bibliotecas
import pandas as pd
import requests


## Campeonato brasileiro 2024

In [79]:
seriaA2024 = requests.get('https://pt.wikipedia.org/wiki/Campeonato_Brasileiro_de_Futebol_de_2024_-_S%C3%A9rie_A')

# Informações do campeonato de 2024
tabelas2024 = pd.read_html(seriaA2024.text)
tabela_classificação2024 = tabelas2024[6]
tabela_jogos2024 = tabelas2024[7]

C:\Users\Thiago\AppData\Local\Temp\ipykernel_13464\3065207342.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tabelas2024 = pd.read_html(seriaA2024.text)


In [86]:
#criando uma lista com os nomes dos times
nomes_times_2024 = list(tabela_jogos2024["Casa \ Fora"])
#criando uma lista com as siglas dos times
sigla_times_2024 = list(tabela_jogos2024.columns)
sigla_times_2024.pop(0)

#criando um dicionário com a sigla dos times
apelidos_times_2024 = dict(zip(sigla_times_2024, nomes_times_2024))


In [88]:
# Ajustar a tabela de jogos
tabela_jogos2024_ajustada = tabela_jogos2024.set_index("Casa \ Fora")
# Desempilhar a tabela
tabela_jogos2024_ajustada = tabela_jogos2024_ajustada.unstack().reset_index()
# Renomear as colunas
tabela_jogos2024_ajustada = tabela_jogos2024_ajustada.rename(columns={"level_0": "Fora", "Casa \ Fora": "Casa", 0: "Placar"})

# Função para ajustar o nome dos times
def ajustar_apelido(linha, apelido):
    time = linha["Fora"]
    return apelido[time]

# Ajustar o nome dos times aplicando a função
tabela_jogos2024_ajustada["Fora"] = tabela_jogos2024_ajustada.apply(ajustar_apelido, axis=1, args=(apelidos_times_2024,))
# Remover jogos entre o mesmo time
tabela_jogos2024_ajustada = tabela_jogos2024_ajustada[tabela_jogos2024_ajustada["Fora"] != tabela_jogos2024_ajustada["Casa"]]


In [100]:
tabela_jogos2024_ajustada["Placar"] = tabela_jogos2024_ajustada["Placar"].fillna("A jogar")

jogos_realizados = tabela_jogos2024_ajustada[tabela_jogos2024_ajustada["Placar"].str.contains("–")]
jogos_faltantes = tabela_jogos2024_ajustada[~tabela_jogos2024_ajustada["Placar"].str.contains("–")]
jogos_faltantes = jogos_faltantes.drop(columns=["Placar"])

# Separando a coluna Placar em gols_casa e gols_fora
jogos_realizados["gols_casa"] = jogos_realizados["Placar"].str.split("–").str[0].astype(int)
jogos_realizados["gols_fora"] = jogos_realizados["Placar"].str.split("–").str[1].astype(int)
jogos_realizados = jogos_realizados.drop(columns=["Placar"])

# Media de gols feitos dentro de casa de um time
# Essa Tabela diz a media de gols que o time fez em casa e a media de gols que o time sofreu dentro de casa
media_gols_casa = jogos_realizados.groupby("Casa").mean(numeric_only=True)
media_gols_casa = media_gols_casa.rename(columns={"gols_casa": "gols feitos", "gols_fora": "gols sofridos"})

# Media de gols feitos e sofridos fora de casa de um time
media_gols_fora = jogos_realizados.groupby("Fora").mean(numeric_only=True)
media_gols_fora = media_gols_fora.rename(columns={"gols_casa": "gols sofridos", "gols_fora": "gols feito"})


C:\Users\Thiago\AppData\Local\Temp\ipykernel_13464\4097725298.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jogos_realizados["gols_casa"] = jogos_realizados["Placar"].str.split("–").str[0].astype(int)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_13464\4097725298.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jogos_realizados["gols_fora"] = jogos_realizados["Placar"].str.split("–").str[1].astype(int)


## Campeonato Brasileiro 2023

In [104]:
seriaA2024 = requests.get('https://pt.wikipedia.org/wiki/Campeonato_Brasileiro_de_Futebol_de_2023_-_S%C3%A9rie_A')

tabelas2023 = pd.read_html(seriaA2024.text)
tabela_jogos = tabelas2023[6]

nomes_times = list(tabela_jogos["Casa \ Fora"])
sigla_times = list(tabela_jogos.columns)
sigla_times.pop(0)

apelidos_times = dict(zip(sigla_times, nomes_times))

tabela_jogos_ajustada = tabela_jogos.set_index("Casa \ Fora")
tabela_jogos_ajustada = tabela_jogos_ajustada.unstack().reset_index()
tabela_jogos_ajustada = tabela_jogos_ajustada.rename(columns={"level_0": "Fora", "Casa \ Fora": "Casa", 0: "Placar"})

tabela_jogos_ajustada["Fora"] = tabela_jogos_ajustada.apply(ajustar_apelido, axis=1, args=(apelidos_times,))
tabela_jogos_ajustada = tabela_jogos_ajustada[tabela_jogos_ajustada["Casa"] != tabela_jogos_ajustada["Fora"]]

tabela_jogos_ajustada["gols_casa"] = tabela_jogos_ajustada["Placar"].str.split("–").str[0].astype(int)
tabela_jogos_ajustada["gols_fora"] = tabela_jogos_ajustada["Placar"].str.split("–").str[1].astype(int)
tabela_jogos_ajustada = tabela_jogos_ajustada.drop(columns=["Placar"])  

media_gols_casa_2023 = tabela_jogos_ajustada.groupby("Casa").mean(numeric_only=True)
media_gols_casa_2023 = media_gols_casa_2023.rename(columns={"gols_casa": "gols feitos", "gols_fora": "gols sofridos"})

media_gols_fora_2023 = tabela_jogos_ajustada.groupby("Fora").mean(numeric_only=True)
media_gols_fora_2023 = media_gols_fora_2023.rename(columns={"gols_fora": "gols feitos", "gols_casa": "gols sofridos"})

# Removendo os times rebaixados
media_gols_casa_2023 = media_gols_casa_2023.drop(index=["Santos", "Goiás", "Coritiba", "América Mineiro"])
media_gols_fora_2023 = media_gols_fora_2023.drop(index=["Santos", "Goiás", "Coritiba", "América Mineiro"])

C:\Users\Thiago\AppData\Local\Temp\ipykernel_13464\3708212160.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tabelas2023 = pd.read_html(seriaA2024.text)
